In [20]:
import numpy as np
import pandas as pd
import os

# Load Data
filepath = "/kaggle/input/california-housing-prices/housing.csv"
df = pd.read_csv(filepath)

# Drop missing values
df.dropna(inplace=True)

# One-hot encoding for categorical feature
df = pd.get_dummies(df, columns=['ocean_proximity'], drop_first=True)

# Ensure only 9 features
selected_features = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 
                     'population', 'households', 'median_income', 'ocean_proximity_INLAND'] 
x = df[selected_features].values.astype(float)
y = df['median_house_value'].values.reshape(-1, 1).astype(float)

# Normalize data
x = (x - x.mean(axis=0)) / x.std(axis=0)
y = (y - y.mean()) / y.std()
#
splitter = int(0.7 * len(x))
xtrain, xtest = x[:splitter], x[splitter:]
ytrain, ytest = y[:splitter], y[splitter:]
np.random.seed(1)
w1 = np.random.randn(9, 4) * 0.05 
b1 = np.zeros((1, 4))
w2 = np.random.randn(4, 4) * 0.05
b2 = np.zeros((1, 4))
w3 = np.random.randn(4, 1) * 0.05
b3 = np.zeros((1, 1))

# Activation functions
def relu(z):
    return np.maximum(0, z)

def reluDifferentiation(z):
    return np.where(z > 0, 1, 0)

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoidDifferentiation(z):
    return sigmoid(z) * (1 - sigmoid(z))

# User chooses activation function
ans = int(input("Choose 1 for ReLU or 2 for Sigmoid: "))
if ans == 1:
    activationfunc = relu
    activationDiff = reluDifferentiation
elif ans == 2:
    activationfunc = sigmoid
    activationDiff = sigmoidDifferentiation
else:
    print("Incorrect option entered, defaulting to ReLU.")
    activationfunc = relu
    activationDiff = reluDifferentiation

# Mean Squared Error function
def MSE(y, ypred):
    return np.mean((y - ypred) ** 2)

# Forward pass
def forwardPass(x, w1, b1, w2, b2, w3, b3):
    z1 = np.dot(x, w1) + b1
    v1 = activationfunc(z1)
    z2 = np.dot(v1, w2) + b2
    v2 = activationfunc(z2)
    z3 = np.dot(v2, w3) + b3  # No activation for output
    return z1, v1, z2, v2, z3

def backProp(x, y, z1, v1, z2, v2, z3, w1, b1, w2, b2, w3, b3, learningRate):
    dz3 = z3 - y
    dw3 = np.dot(v2.T, dz3) / len(y)
    db3 = np.sum(dz3, axis=0, keepdims=True) / len(y)
    
    dv2 = np.dot(dz3, w3.T)
    dz2 = dv2 * activationDiff(z2)
    
    dw2 = np.dot(v1.T, dz2) / len(y)
    db2 = np.sum(dz2, axis=0, keepdims=True) / len(y)
    
    dv1 = np.dot(dz2, w2.T)
    dz1 = dv1 * activationDiff(z1)
    
    dw1 = np.dot(x.T, dz1) / len(y)
    db1 = np.sum(dz1, axis=0, keepdims=True) / len(y)

    # Update weights and biases correctly
    w1 -= learningRate * dw1
    b1 -= learningRate * db1
    w2 -= learningRate * dw2
    b2 -= learningRate * db2
    w3 -= learningRate * dw3
    b3 -= learningRate * db3

    return w1, b1, w2, b2, w3, b3


# User-defined stopping criteria
epochs = int(input("Enter number of epochs: "))
error_threshold = float(input("Enter error threshold (or -1 to ignore): "))

batch_size = 64
learningRate = 0.05

for epoch in range(epochs):
    shuffled_indices = np.random.permutation(len(xtrain))
    xtrain_shuffled = xtrain[shuffled_indices]
    ytrain_shuffled = ytrain[shuffled_indices]

    for i in range(0, len(xtrain), batch_size):
        x_batch = xtrain_shuffled[i:i+batch_size]
        y_batch = ytrain_shuffled[i:i+batch_size]

        z1, v1, z2, v2, z3 = forwardPass(x_batch, w1, b1, w2, b2, w3, b3)
        w1, b1, w2, b2, w3, b3 = backProp(x_batch, y_batch, z1, v1, z2, v2, z3, w1, b1, w2, b2, w3, b3, learningRate)

    _, _, _, _, ypred_train = forwardPass(xtrain, w1, b1, w2, b2, w3, b3)
    loss = MSE(ytrain, ypred_train)
    if epoch % 5 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")
        print("Weights w1:", w1)
    if error_threshold > 0 and loss < error_threshold:
        print(f"Stopping early at epoch {epoch} because loss < {error_threshold}")
        break

_, _, _, _, ypred_test = forwardPass(xtest, w1, b1, w2, b2, w3, b3)
test_loss = MSE(ytest, ypred_test)
print(f"Final Test Loss: {test_loss:.4f}")


Choose 1 for ReLU or 2 for Sigmoid:  1
Enter number of epochs:  10
Enter error threshold (or -1 to ignore):  0.075


Epoch 0, Loss: 0.9461
Weights w1: [[ 0.08309174 -0.0276209  -0.0237205  -0.05380623]
 [ 0.03883485 -0.12095505  0.08157355 -0.03596465]
 [ 0.01672314 -0.00993799  0.07307438 -0.10405736]
 [-0.01343204 -0.01462636  0.06088526 -0.05593358]
 [-0.00729273 -0.04389868  0.00472759  0.02879483]
 [-0.05626685  0.05275662  0.04596636  0.02517324]
 [ 0.04670173 -0.03391759 -0.00316195 -0.04737264]
 [ 0.002644    0.05655764 -0.02438732 -0.02516137]
 [-0.04224531 -0.05225682 -0.04091163  0.00461032]]
Epoch 5, Loss: 0.2999
Weights w1: [[ 0.02874324 -0.33508588 -0.01148481  0.00143726]
 [-0.16850099 -0.32662785 -0.07234724  0.06353222]
 [ 0.08999216  0.05408669  0.17191447  0.1317297 ]
 [-0.07647902  0.02763473  0.08809996 -0.09729892]
 [ 0.07634031  0.11135994  0.0715449  -0.11546304]
 [-0.17295135 -0.55961763 -0.03232328  0.04378057]
 [ 0.20308706  0.2274649   0.11275567 -0.17435969]
 [ 0.05187671  0.47893759  0.17407127 -0.1407546 ]
 [-0.24256402 -0.01390975 -0.09667876  0.15224593]]
Final Test L